# Instalação do Z3/Pandas

In [ ]:
!pip install z3-solver pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 26.0 MB/s eta 0:00:00


## Iniciando o projeto
### Importação das bibliotecas e recursos necessários


*   Importando um dataframe por meio de uma url
*   Iniciando o solver do Z3



In [ ]:
import pandas as pd
from z3 import *

CSV_URL = 'https://raw.githubusercontent.com/thiagoalvesifce/logicomp/master/vertebral_column_features_binarized_abnormal_no.csv'
solver = Solver()

df = pd.read_csv(CSV_URL)

num_colunas = df.shape[1]
# k representa o numero de colunas/features, que vai representar a quantidade de variáveis (x1, x2, ..., xk)
k = num_colunas - 1

### Fazendo um map no df para padronizar tudo em 0 ou 1
#### A ultima coluna class P (indica que o paciente tem patologia) e NO (indica que o paciente não tem patologia)

*   p -> 1
*   NO -> 0



In [ ]:
df['class'] = (df['class'].replace({'P': 1, 'NO': 0})).astype(int)

/tmp/ipython-input-23-509452464.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['class'] = (df['class'].replace({'P': 1, 'NO': 0})).astype(int)


**Divisão dos dados**
* Selecionar aleatoriamente 90% dos pacientes com patologia e
90% dos pacientes sem patologia para compor o conjunto de
treinamento;
* Os 10% restantes de cada grupo formarão o conjunto de teste.

In [ ]:
df_with_pathologies = df[ df['class'] == 1 ]
df_without_pathologies = df[ df['class'] == 0 ]

train_with_pathologies =  df_with_pathologies.sample(frac=0.9, random_state=42)
train_without_pathologies = df_without_pathologies.sample(frac=0.9, random_state=42)

test_with_pathologies = df_with_pathologies.drop( train_with_pathologies.index )
test_without_pathologies = df_without_pathologies.drop( train_without_pathologies.index )

train = pd.concat([train_with_pathologies, train_without_pathologies]).sample(frac=1, random_state=42)
test  = pd.concat([test_with_pathologies,  test_without_pathologies ]).sample(frac=1, random_state=42)

labels = train['class']
labels_test = test['class']

Criação de Variáveis para indicar o número máximo de regras (M) e o número de literais (L)

In [ ]:
M = int(input('Enter the number of rules: '))
L = int(input('Enter the number of literals: '))

Enter the number of rules: 7
Enter the number of literals: 4


## Criação das variáveis (p, n, s)
Cada coluna do dataframe se tornará uma variável e terão suas respectivas representações de (p, n, s) no solver do z3.

*   As variáveis p: representam a positividade da variável na regra
*   As variáveis n: representam a negatividade da variável na regra
*   As variáveis s: representam a "invisibilidade/(don's care)" da variável na regra

Além da indicação de qual coluna é a variável, é necessário saber também a qual regra aquela variável pertence. No caso, temos M regras.

A representação (exemplo):

1. p_1_1: "A variável x1 tem valor positivo na regra 1"
2. n_1_1: "A variável x1 tem valor negativo na regra 1"
3. s_1_1: "A variável x1 não aparece/não tem importância na regra 1"

In [ ]:
p = {(i, j): Bool(f"p_{i}_{j}") for i in range(1, k+1) for j in range(1,M+1)}
n = {(i, j): Bool(f"n_{i}_{j}") for i in range(1, k+1) for j in range(1, M+1)}
s = {(i, j): Bool(f"s_{i}_{j}") for i in range(1, k+1) for j in range(1, M+1)}

print(M)

7


## ✅ Restrições: Representação Exclusiva das Variáveis

Para cada **variável** em **cada regra**, ela deve aparecer **exatamente** em uma das seguintes formas:

- **Positiva** — `p_{i,j}`  
- **Negativa** — `n_{i,j}`  
- **Não aparece** (*don’t care*) — `s_{i,j}`  

---

### 🔸 Condições a serem impostas

1. **Pelo menos uma** deve ser verdadeira:
   ```z3
   p_{i,j} ∨ n_{i,j} ∨ s_{i,j}
   ```
2. **No máximo uma** deve ser verdadeira:
   ```z3
   ¬(p_{i,j} ∧ n_{i,j}) ∧ ¬(p_{i,j} ∧ s_{i,j}) ∧ ¬(n_{i,j} ∧ s_{i,j})
   ```

In [ ]:
for i in range(1, k+1):
    for j in range(1, M+1):
        solver.add( Or(p[(i,j)], n[(i,j)], s[(i,j)]) )
        solver.add( Not(And(p[(i,j)], n[(i,j)])) )
        solver.add( Not(And(p[(i,j)], s[(i,j)])) )
        solver.add( Not(And(n[(i,j)], s[(i,j)])) )

## ✅ Restrição: Máximo de Literais por Regra

Cada **regra** (ou termo) da DNF pode usar no máximo **L** literais (positivas ou negativas).

---

### 🔸 Objetivo

Assegurar que nenhuma cláusula individual fique muito complexa, limitando o número de literais em cada termo a **L**.


In [ ]:
# Como não há como somar valores booleanos do z3, fazemos uma função auxiliar que retorna 1 ou 0 (inteiros) para executar a soma:

def Bool2Int(b):
    return If(b, 1, 0)

for j in range(1, M+1):
    literais_na_regra = []
    for i in range(1, k+1):
        # Se a regra j usa a feature i (positiva ou negativa), 0 caso contrário
        literais_na_regra.append(
            Bool2Int( Or(p[(i,j)], n[(i,j)]) )
        )
    # Impor: soma de literais na regra j ≤ L
    solver.add( Sum(literais_na_regra) <= L )
print(L)

4


## ✅ Restrições: Proibir Regra Completamente “Vazia”
Para cada **regra** `j` é preciso garantir que **pelo menos uma literal** apareça (ou seja, não seja `s_{i,j}` para todo `i`).

* Se `s_{i,j} = True` significa “feature i não aparece”.
* Impomos: para cada j, existe algum i tal que `¬(s_{i,j})` seja True → ou seja, p ou n

In [ ]:
for j in range(1, M+1):
    claus = []
    for i in range(1, k+1):
        claus.append( Not(s[(i,j)]) )
    solver.add( Or(*claus) )

## 🎯 Objetivo Geral  
Para cada linha do conjunto de dados, queremos garantir que a solução **rejeite** os exemplos negativos (classe 0) e **aceite** os exemplos positivos (classe 1), usando variáveis booleanas que representam literais positivas (`p`), negativas (`n`) ou “don’t-care” (`s`).

---

## 🔴 Exemplos Negativos (classe 0)  

**Aplicação**:  
   - Para cada termo (regra) _j_:  
     - Deve haver **ao menos um literal** que “quebre” a regra para esta linha.  
     - Se uma feature vale **1**, então forçamos o literal negativo nessa regra.  
     - Se a feature vale **0**, forçamos o literal positivo.  


---

## 🟢 Exemplos Positivos (classe 1)

**Aplicação**:  
   - Para cada termo _j_:  
     - Construímos uma lista de **implicações**:  
       - Se o literal positivo estiver ativo (`p_{i,j}`), então a feature deve valer 1.  
       - Se o literal negativo estiver ativo (`n_{i,j}`), então a feature deve valer 0.  
       - Literais “don’t-care” (`s_{i,j}`) não impõem nenhuma condição.

In [ ]:
for idx, linha in train.iloc[:, :-1].iterrows():
    valor = int(labels[idx])  # 0 ou 1
    if valor == 0:
        for j in range(1, M+1):
            falha_disj = []
            for i in range(0, k):
                if int(linha.values[i]) == 1:
                    falha_disj.append( n[(i+1, j)] )
                else:
                    falha_disj.append( p[(i+1, j)] )
            solver.add( Or(*falha_disj) )
    else:
        regras_aceitam = []
        for j in range(1, M+1):
            conds = []
            for i in range(0, k):
                # feature equivale a cada coluna daquela linha especifica
                valor_feat = int(linha.values[i])
                conds.append( Implies(p[(i+1, j)], valor_feat == 1) )
                conds.append( Implies(n[(i+1, j)], valor_feat == 0) )
            regras_aceitam.append( And(*conds) )
        solver.add( Or(*regras_aceitam) )

## 1. Verificação de Satisfatibilidade
- **Objetivo:** Checar se existe um modelo (atribuição de variáveis booleanas) que satisfaça todas as restrições impostas ao solver.
---
## 2. Extração do Modelo e Montagem das Regras
- **Objetivo:** Traduzir o modelo retornado pelo solver em um conjunto de regras lógicas.
---
## 3. Função de Predição (`predict`)
- **Objetivo:** Classificar uma nova amostra com base no conjunto de regras extraídas.
---
## 4. Cálculo da Acurácia no Conjunto de Teste
- **Objetivo:** Medir a qualidade do modelo comparando predições com rótulos reais.

In [ ]:
sat_result = solver.check()
print(solver)
if sat_result == sat:
    print(f"\n==> SOLUÇÃO ENCONTRADA: R = {M} regras, L = {L} literais")
    modelo = solver.model()

    rules = []
    for j in range(1, M+1):
        rule = []
        for i in range(1, k+1):
            if is_true(modelo.evaluate(p[(i, j)])):
                rule.append((i, 1))
            elif is_true(modelo.evaluate(n[(i, j)])):
                rule.append((i, 0))
        rules.append(rule)

    for j, rule in enumerate(rules, start=1):
        if rule:
            lits = [f"(x{i} == {val})" for i, val in rule]
            print(f"Regra {j}: {' ∧ '.join(lits)}")
        else:
            print(f"Regra {j}: (nenhuma feature → termo vazio)")

    # 4) Definir função de predição
    def predict(x, rules):
        for rule in rules:
            ok = True
            for i, val in rule:
                # checa cada literal
                if val == 1 and x[i-1] != 1:
                    ok = False
                    break
                if val == 0 and x[i-1] != 0:
                    ok = False
                    break
            if ok:
                return 1
        return 0

    correct = 0
    total = len(test)
    for idx, row in test.iloc[:, :-1].iterrows():
        x = row.values.astype(int)
        y_true = int(labels_test[idx])
        y_pred = predict(x, rules)
        if y_pred == y_true:
            correct += 1

    acuracia = correct / total
    print(f"\nAcurácia no teste: {acuracia:.2%}")
else:
    print("Não existe classificação perfeita dentro dos limites testados.")


[Or(p_1_1, n_1_1, s_1_1),
 Not(And(p_1_1, n_1_1)),
 Not(And(p_1_1, s_1_1)),
 Not(And(n_1_1, s_1_1)),
 Or(p_1_2, n_1_2, s_1_2),
 Not(And(p_1_2, n_1_2)),
 Not(And(p_1_2, s_1_2)),
 Not(And(n_1_2, s_1_2)),
 Or(p_1_3, n_1_3, s_1_3),
 Not(And(p_1_3, n_1_3)),
 Not(And(p_1_3, s_1_3)),
 Not(And(n_1_3, s_1_3)),
 Or(p_1_4, n_1_4, s_1_4),
 Not(And(p_1_4, n_1_4)),
 Not(And(p_1_4, s_1_4)),
 Not(And(n_1_4, s_1_4)),
 Or(p_1_5, n_1_5, s_1_5),
 Not(And(p_1_5, n_1_5)),
 Not(And(p_1_5, s_1_5)),
 Not(And(n_1_5, s_1_5)),
 Or(p_1_6, n_1_6, s_1_6),
 Not(And(p_1_6, n_1_6)),
 Not(And(p_1_6, s_1_6)),
 Not(And(n_1_6, s_1_6)),
 Or(p_1_7, n_1_7, s_1_7),
 Not(And(p_1_7, n_1_7)),
 Not(And(p_1_7, s_1_7)),
 Not(And(n_1_7, s_1_7)),
 Or(p_2_1, n_2_1, s_2_1),
 Not(And(p_2_1, n_2_1)),
 Not(And(p_2_1, s_2_1)),
 Not(And(n_2_1, s_2_1)),
 Or(p_2_2, n_2_2, s_2_2),
 Not(And(p_2_2, n_2_2)),
 Not(And(p_2_2, s_2_2)),
 Not(And(n_2_2, s_2_2)),
 Or(p_2_3, n_2_3, s_2_3),
 Not(And(p_2_3, n_2_3)),
 Not(And(p_2_3, s_2_3)),
 Not(And(n_2_3,